In [115]:
import numpy as np
import pandas as pd
import random
from itertools import product

In [116]:
df=pd.read_csv('wr2.csv')

In [117]:
def splitt(str):
    return str.split('+')[-1]

In [118]:
df=pd.read_csv('wr2.csv')
df["METRIC NAME"]=df["METRIC NAME"].apply(lambda x:splitt(x))
df.to_csv('NEW_RATE.csv')
df.head(5)

,ANSWER ID,METRIC NAME,COMPANY NAME,YEAR,VALUE
0,2233010.0,Gross Profit,Danaher,2012,9414300000
1,2233013.0,Gross Profit,Danaher,2015,10762500000
2,2233016.0,Gross Profit,Danaher,2014,9892600000
3,2233001.0,Gross Profit,Danaher,2011,8176600000
4,2233004.0,Gross Profit,Danaher,2010,6404500000


In [119]:
def missing_value(df,year,company):
    
    #Get metric index table
    METRIC_TABLE=df[['METRIC NAME']].drop_duplicates().sort_values(by='METRIC NAME').reset_index(drop=True)
    METRIC_TABLE['METRIC ID']= pd.Categorical(METRIC_TABLE['METRIC NAME'].astype(str)).codes
    #METRIC_TABLE.to_csv('Metric_index_tabel.csv',index=False)
    
    #Get company index table
    COMPANY_TABLE=df[['COMPANY NAME']].drop_duplicates().sort_values(by='COMPANY NAME').reset_index(drop=True)
    COMPANY_TABLE['COMPANY ID'] = pd.Categorical(COMPANY_TABLE['COMPANY NAME'].astype(str)).codes
    ###------pseudo email address
    COMPANY_TABLE['COMPANY EMAIL']=COMPANY_TABLE['COMPANY NAME'].apply(lambda x:'{}@gmail.com'.format(x.lower().replace(' ','')))
    #COMPANY_TABLE.to_csv('Company_index_tabel.csv',index=False)
    
    #Get Survey Question Table
    survey=df[['METRIC NAME']].drop_duplicates().reset_index(drop=True)
    ###------pseudo survey questions
    survey['SURVEY QUESTIONS']=survey['METRIC NAME'].apply(lambda x:'Do you have answer to {}? If yes, please specify; if no,please leave the question blank'.format(x))
    #survey.to_csv('Survey_index_table.csv',index=False)
    
    #Generate new dataframe
    full=pd.DataFrame(list(product([company], METRIC_TABLE['METRIC NAME'].unique(),[year])),
                      columns=['COMPANY NAME','METRIC NAME','YEAR'])
    
    #merge with df for wanted info
    full=full.merge(df.drop('ANSWER ID',1),on=['COMPANY NAME','METRIC NAME','YEAR'],how='left')
    
    #return only observations with missing values
    selected=full[ ((full.VALUE.isnull()) | (full.VALUE == 'Unclear') | (full.VALUE == 'Unknown')) & (full.YEAR==year)
                 & (full['COMPANY NAME']==company)]
    selected=selected[['COMPANY NAME', 'YEAR','METRIC NAME']].sort_values(by=['COMPANY NAME', 'YEAR'])
    
    #Get survey questions for those missing value
    selected=selected.merge(survey,on='METRIC NAME',how='left')
    
    #def questionid
    def questionid(row):
        company_id=COMPANY_TABLE.loc[COMPANY_TABLE['COMPANY NAME']==row['COMPANY NAME'],'COMPANY ID'].values[0]
        metric_id=METRIC_TABLE.loc[METRIC_TABLE['METRIC NAME']==row['METRIC NAME'],'METRIC ID'].values[0]
        year_id=row.YEAR
        return int(str(year_id)+format(company_id,'04')+format(metric_id,'04'))
    
    #Generate questionid for each question
    selected['QUESTIONID']=selected.apply(questionid,axis=1)
    
    return selected

### Test

In [121]:
table0=missing_value(df,2016,'Danaher')
table0.head()

,COMPANY NAME,YEAR,METRIC NAME,SURVEY QUESTIONS,QUESTIONID
0,Danaher,2016,% CSPO Used,"Do you have answer to % CSPO Used? If yes, ple...",201607530000
1,Danaher,2016,Absenteeism rate,Do you have answer to Absenteeism rate? If yes...,201607530001
2,Danaher,2016,Access to Childcare Services,Do you have answer to Access to Childcare Serv...,201607530002
3,Danaher,2016,Access to Healthcare for Non-work-related Inci...,Do you have answer to Access to Healthcare for...,201607530003
4,Danaher,2016,Access to Healthcare for Work-related Incidents,Do you have answer to Access to Healthcare for...,201607530004


In [ ]:
question1 = list(table0["SURVEY QUESTIONS"])
question0=["open_ended"]
questions_list=[[x,y] for x in question0 for y in question1]
questions_list[0:5]

In [123]:
survey_title = 'Test Survey'
data = [['dingrenjie12345@gmail.com','Renjie','Ding'],['yuanz15@wfu.edu','Y','C']]
recipients_df = pd.DataFrame(data, columns=["email", "first_name", "last_name"])

results0=process(survey_title, questions_list, recipients_df)
print(results0)

survey id: 1016


KeyError: 'data'

In [ ]:
result_answer=after_survey(results0)

In [54]:
new_table=pd.DataFrame()
new_question=[]
new_answer=[]

for i in range(8):
    new_question.append(table0["SURVEY QUESTIONS"][i])
for i in result_answer[0]:
    new_answer.append(result_answer[0][i])
print(new_answer)
new_answer=new_answer[len(new_answer)-8:]
print(new_answer)

new_table["Company"]=list(["Danaher","Danaher","Danaher","Danaher","Danaher","Danaher","Danaher","Danaher"])
new_table["Question"]=new_question
new_table["Answer"]=new_answer

['157587069', '10201416351', '216378077', '2018-09-06T21:52:21+00:00', '2018-09-06T21:53:02+00:00', '144.121.68.19', 'dingrenjie12345@gmail.com', 'Renjie', 'Ding', '80', 'yes', '987542', '75', '98', '87', '95', '56']
['80', 'yes', '987542', '75', '98', '87', '95', '56']
